# Microsoft Entra Agent ID + LangGraph

This notebook creates an Entra Agent ID access token (T2) and uses it inside a LangGraph agent tool to call Microsoft Graph.

**What you will do:**
- Set up Entra Agent ID (Blueprint + Agent Identity)
- Store the required IDs/secrets in environment variables
- Use a LangGraph agent tool to call Microsoft Graph with the agent token

## Azure / Entra setup (one-time)

The steps below are adapted from the 3P-Agent-ID-Demo lab guide.

### Prerequisites
- Azure CLI
- PowerShell 7.5+
- Microsoft Graph PowerShell SDK: `Install-Module Microsoft.Graph -Scope CurrentUser`
- Entra role: Global Administrator, Cloud Application Administrator, or Application Administrator

### Authenticate
```bash
az login --use-device-code --tenant <your-tenant-id>
```
```powershell
pwsh
$tenantId = (az account show --query tenantId -o tsv)
Connect-MgGraph -Scopes "AgentIdentityBlueprint.AddRemoveCreds.All","AgentIdentityBlueprint.Create","DelegatedPermissionGrant.ReadWrite.All","Application.Read.All","AgentIdentityBlueprintPrincipal.Create","AppRoleAssignment.ReadWrite.All","User.Read" -TenantId $tenantId
Get-MgContext
```

### Create Blueprint + Agent Identity (automated workflow)
```powershell
# From the 3P-Agent-ID-Demo repo root
. ./EntraAgentID-Functions.ps1
$result = Start-EntraAgentIDWorkflow -Permissions @("User.Read.All")

# Capture these values for the notebook env vars:
$result.Connection.TenantId
$result.Blueprint.BlueprintAppId
$result.Blueprint.ClientSecret
$result.Agent.AgentIdentityAppId
```

### Required environment variables
Create a .env file (or set environment variables) with:
```text
ENTRA_TENANT_ID=...
ENTRA_BLUEPRINT_APP_ID=...
ENTRA_BLUEPRINT_CLIENT_SECRET=...
ENTRA_AGENT_APP_ID=...

AZURE_OPENAI_ENDPOINT=...
AZURE_OPENAI_API_KEY=...
AZURE_OPENAI_API_VERSION=...
AZURE_OPENAI_DEPLOYMENT=...
```

In [ ]:
# Optional: install dependencies
%pip install -q langgraph langchain-openai langchain-core python-dotenv requests

import os
import requests
from dotenv import load_dotenv

load_dotenv()

REQUIRED_ENV = [
    "ENTRA_TENANT_ID",
    "ENTRA_BLUEPRINT_APP_ID",
    "ENTRA_BLUEPRINT_CLIENT_SECRET",
    "ENTRA_AGENT_APP_ID",
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_DEPLOYMENT",
]

missing = [key for key in REQUIRED_ENV if not os.getenv(key)]
if missing:
    raise EnvironmentError(f"Missing environment variables: {', '.join(missing)}")

TENANT_ID = os.getenv("ENTRA_TENANT_ID")
BLUEPRINT_APP_ID = os.getenv("ENTRA_BLUEPRINT_APP_ID")
BLUEPRINT_CLIENT_SECRET = os.getenv("ENTRA_BLUEPRINT_CLIENT_SECRET")
AGENT_APP_ID = os.getenv("ENTRA_AGENT_APP_ID")

TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

In [ ]:
def get_t1_token():
    data = {
        "client_id": BLUEPRINT_APP_ID,
        "client_secret": BLUEPRINT_CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "api://AzureADTokenExchange/.default",
        "fmi_path": AGENT_APP_ID,
    }
    resp = requests.post(TOKEN_URL, data=data)
    resp.raise_for_status()
    return resp.json()["access_token"]

def get_t2_token(t1_token: str):
    data = {
        "client_id": AGENT_APP_ID,
        "grant_type": "client_credentials",
        "scope": "https://graph.microsoft.com/.default",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": t1_token,
    }
    resp = requests.post(TOKEN_URL, data=data)
    resp.raise_for_status()
    return resp.json()["access_token"]

def get_agent_token():
    t1 = get_t1_token()
    return get_t2_token(t1)

In [ ]:
def graph_get(path: str, access_token: str):
    url = f"https://graph.microsoft.com/v1.0/{path.lstrip('/')}"
    headers = {"Authorization": f"Bearer {access_token}"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

# Quick sanity check (requires User.Read.All or Directory.Read.All on the Agent Identity)
token = get_agent_token()
graph_get("users?$top=5", token)

In [ ]:
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent

@tool
def list_users(top: int = 5) -> str:
    """List users from Microsoft Graph using an Entra Agent ID token."""
    token = get_agent_token()
    data = graph_get(f"users?$top={top}", token)
    users = data.get("value", [])
    lines = [f"{u.get('displayName','')} <{u.get('userPrincipalName','')}>" for u in users]
    return "\n".join(lines) if lines else "No users returned."

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0,
)

agent = create_agent(
    model=llm,
    tools=[list_users],
    prompt="You are a helpful agent that can list directory users.",
)

In [ ]:
query = "List 5 users from the directory."
result = agent.invoke({"messages": [("user", query)]})
result["messages"][-1].content

## How to run
1. Complete the Azure / Entra setup above and capture the IDs/secrets.
2. Set the environment variables (or a .env file) for Entra Agent ID and Azure OpenAI.
3. Run the notebook cells top-to-bottom.

If the Graph call fails with `Authorization_RequestDenied`, wait a few minutes and request a new T2 token (permissions can take time to propagate).